In [2]:
# connect to postgres
from spark_session import spark

In [6]:
jdbc_configs = {
    "host":"localhost",
    "port":"5432",
    "user":"jarvis",
    "password":"2650"
}

jdbc_url = 'jdbc:postgresql://localhost:5432/avengers'

# spark

In [78]:
from pyspark.sql.types import StructType,StructField,IntegerType,TimestampType,StringType

records_schema = StructType([
    StructField("id",IntegerType()),
    StructField("unique_key",StringType()),
    StructField("value",StringType()),
    StructField("updated_at",TimestampType())
])

table='records'

tgt_file = f'./target_files/{table}'
try:
    tgt_tbl = spark.read.parquet(tgt_file)
    src = spark.read.jdbc(url=jdbc_url,table=table,properties=jdbc_configs)
    tgt_timestamp = tgt_tbl.selectExpr("max(updated_at) as max_updated_at")
    src = src.alias("src")
    tgt = tgt_timestamp.alias("tgt")
    filtered = src.crossJoin(tgt).select("src.*").where("src.updated_at > tgt.max_updated_at")
    filtered.write.mode("append").option("header",True).parquet(f'{tgt_file}')
    # tgt.show()
    filtered.show()

except Exception as e:
    print(e)


25/06/30 08:31:44 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
25/06/30 08:31:44 INFO SparkContext: Starting job: parquet at <unknown>:0
25/06/30 08:31:44 INFO DAGScheduler: Got job 81 (parquet at <unknown>:0) with 1 output partitions
25/06/30 08:31:44 INFO DAGScheduler: Final stage: ResultStage 89 (parquet at <unknown>:0)
25/06/30 08:31:44 INFO DAGScheduler: Parents of final stage: List()
25/06/30 08:31:44 INFO DAGScheduler: Missing parents: List()
25/06/30 08:31:44 INFO DAGScheduler: Submitting ResultStage 89 (MapPartitionsRDD[244] at parquet at <unknown>:0), which has no missing parents
25/06/30 08:31:44 INFO MemoryStore: Block broadcast_107 stored as values in memory (estimated size 104.8 KiB, free 7.8 GiB)
25/06/30 08:31:44 INFO MemoryStore: Block broadcast_107_piece0 stored as bytes in memory (estimated size 37.9 KiB, free 7.8 GiB)
25/06/30 08:31:44 INFO BlockManagerInfo: Added broadcast_107_piece0 in memory on localhost:44471 (size: 37.9 KiB, free: 7.8 Gi

+---+----------+-----+----------+
| id|unique_key|value|updated_at|
+---+----------+-----+----------+
+---+----------+-----+----------+



25/06/30 08:31:45 INFO JDBCRDD: closed connection
25/06/30 08:31:45 INFO Executor: Finished task 0.0 in stage 97.0 (TID 112). 1508 bytes result sent to driver
25/06/30 08:31:45 INFO TaskSetManager: Finished task 0.0 in stage 97.0 (TID 112) in 30 ms on localhost (executor driver) (1/1)
25/06/30 08:31:45 INFO TaskSchedulerImpl: Removed TaskSet 97.0, whose tasks have all completed, from pool 
25/06/30 08:31:45 INFO DAGScheduler: ResultStage 97 (showString at <unknown>:0) finished in 0.049 s
25/06/30 08:31:45 INFO DAGScheduler: Job 87 is finished. Cancelling potential speculative or zombie tasks for this job
25/06/30 08:31:45 INFO TaskSchedulerImpl: Killing all running tasks in stage 97: Stage finished
25/06/30 08:31:45 INFO DAGScheduler: Job 87 finished: showString at <unknown>:0, took 0.052601 s


25/06/30 08:42:03 INFO BlockManagerInfo: Removed broadcast_117_piece0 on localhost:44471 in memory (size: 7.9 KiB, free: 7.8 GiB)
25/06/30 08:42:03 INFO BlockManagerInfo: Removed broadcast_116_piece0 on localhost:44471 in memory (size: 193.0 B, free: 7.8 GiB)
25/06/30 08:42:03 INFO BlockManagerInfo: Removed broadcast_113_piece0 on localhost:44471 in memory (size: 35.7 KiB, free: 7.8 GiB)


In [ ]:
# src.printSchema()
# df.printSchema()
# src.printSchema()
# assert src.schema == df.schema,"Schema error Triggered"


# tgt.show()

# print(src.schema)
# print(tgt.schema)

25/06/29 22:09:18 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.


In [ ]:
# string error